In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

In [2]:
# Load the dataset
df = pd.read_csv('sensor_data.csv')

# Preprocess the data
# Convert categorical labels ('pothole', 'normal road') to numerical values
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Features and labels
X = df.drop(columns=['time', 'label'])  # Drop 'time' and 'label' columns
y = df['label']

In [3]:
# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33, random_state=42)  # 10% test, 20% validation


In [4]:
# Build the neural network model
model = Sequential()

# Input layer and first hidden layer with 128 neurons, ReLU activation, and L2 regularization
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu', kernel_regularizer=regularizers.l2(0.01)))

# Add Dropout to prevent overfitting
model.add(Dropout(0.5))

# Second hidden layer with 64 neurons
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

# Add Dropout to prevent overfitting
model.add(Dropout(0.5))

# Third hidden layer with 32 neurons
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

# Output layer
model.add(Dense(1, activation='sigmoid'))  # Output is binary (pothole or normal road)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
# Train the model with 30 epochs, large batch size for generalization, and validation data
history = model.fit(X_train, y_train, epochs=30, batch_size=64, validation_data=(X_val, y_val), verbose=1)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy*100:.2f}%")


Epoch 1/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7821 - loss: 2.5790 - val_accuracy: 0.9269 - val_loss: 1.6790
Epoch 2/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9056 - loss: 1.8525 - val_accuracy: 1.0000 - val_loss: 1.2637
Epoch 3/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9149 - loss: 1.4501 - val_accuracy: 1.0000 - val_loss: 1.1289
Epoch 4/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9591 - loss: 1.1995 - val_accuracy: 1.0000 - val_loss: 1.0194
Epoch 5/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9634 - loss: 1.0825 - val_accuracy: 1.0000 - val_loss: 0.9312
Epoch 6/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9857 - loss: 0.9519 - val_accuracy: 1.0000 - val_loss: 0.8562
Epoch 7/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9910 - loss: 0.8747 - val_accuracy: 1.0000 - val_loss: 0.7916
Epoch 8/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9902 - loss: 0.8092 - val_accuracy: 1.0000 - val_loss:

In [7]:
model.save('pothole_detection_model.h5')